In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 18.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 KB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 26.2 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
%cd /content/drive/MyDrive/data/
%ls

/content/drive/.shortcut-targets-by-id/1lTbeLKl99_-ux0hJXW-AtqLq-J_M1fIm/data
compare_5/                 gpt2_predict_model/
finalWords.json            gpt2_predict_model_30/
finetune_eval.txt          gpt2_predict_model_6/
finetune_train.txt         predict_embeddings_model_weights
finetune_val.txt           predict_embeddings_model_weights_30
gpt2_adapter_casual/       predict_embeddings_model_weights_6
gpt2_adapter_mlm_model/    quiz_questions_10.txt
gpt2_adapter_mlm_model_3/  quiz_questions_25.txt
gpt2_casual/               quiz_questions_2.txt
gpt2_casual_2              quiz_questions_50.txt
gpt2_finetune_random/      quiz_questions_5.txt
gpt2_mlm_model/            quiz_questions.txt
gpt2_mlm_model_2/          words.txt
gpt2_mlm_model_3/


In [ ]:
from transformers import pipeline, AutoTokenizer, AutoModelWithLMHead
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import torch
import json
import random
from transformers import AutoTokenizer, GPT2DoubleHeadsModel

In [ ]:
import torch

# If there's a GPU available...
if torch.cuda.is_available():    

    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: Tesla T4


In [ ]:
# Load the JSON file
with open('finalWords.json') as f:
    all_words = json.load(f)

model_name = 'gpt2'

tokenizer = AutoTokenizer.from_pretrained("gpt2")
model = GPT2DoubleHeadsModel.from_pretrained("gpt2")
model = model.to(device)
# Get the current vocabulary size
original_vocab_size = len(tokenizer)
new_word_list = [i.strip().lower() for i in open("words.txt").readlines()]
num_new_words = len(new_word_list)

num_added = tokenizer.add_tokens(new_word_list)

model.resize_token_embeddings(len(tokenizer))

Some weights of GPT2DoubleHeadsModel were not initialized from the model checkpoint at gpt2 and are newly initialized: ['multiple_choice_head.summary.weight', 'multiple_choice_head.summary.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Embedding(100258, 768)

In [ ]:
# print(new_word_list)
print(len(new_word_list))

50001


In [ ]:
num_choices = 25


In [ ]:
import random
import re

with open('finalWords.json') as f:
    all_words = json.load(f)

quiz_questions = []
quiz_answers = []
new_words = [i.strip() for i in open("finetune_eval.txt").readlines()]
new_words_copy = new_words.copy()
# Loop through each word in the JSON file
count = 0
words_dict = {}
for i in all_words:
  words_dict[i['word']] = i['example']
for i in range(0,  len(new_words_copy)):
  word = new_words[i]
  example = words_dict[word]
  if word.lower() not in example.lower():
    continue
  # Get a list of 3 random words to use as distractors
  new_words_copy.remove(word)
  distractors = [word.lower()] + random.sample(new_words_copy, num_choices - 1)
  new_words_copy.append(word)
  # print("example", example, "word", word, "distractors", distractors)
  # Add the correct answer to the beginning of the options list
  # print(word)
  # print(example)
  # print(distractors)
  # print(word)
  options = [re.sub(r'(?i)'+re.escape(word.lower()), word_choice.lower(), example) + " [CLS]" for word_choice in distractors]
  
  # print(options)
  # Add the question and set the answer to 0
  quiz_questions.append(options)
  quiz_answers.append(0)
  count += 1


In [ ]:
for i in range(0, 5):
  print(quiz_questions[i])

In [ ]:
import json
with open(f"quiz_questions_{num_choices}.txt", "w+") as f:
  json.dump(quiz_questions, f)
print(len(quiz_questions))

12211


In [ ]:
import json
with open(f"quiz_questions_{num_choices}.txt") as f:
  quiz_questions = json.load(f)
print(len(quiz_questions))

12211


In [ ]:
import json

num_added_tokens = tokenizer.add_special_tokens({"cls_token": "[CLS]"})
# Update the model embeddings with the new vocabulary size
# print("tokenizer length", len(tokenizer))
embedding_layer = model.resize_token_embeddings(len(tokenizer))

num_choices_arr = [5]
model_ans = []
for num_choices in num_choices_arr:

  with open(f"quiz_questions_{num_choices}.txt") as f:
    quiz_questions = json.load(f)
  print("NUM_CHOICES:", num_choices)

 

  questions = quiz_questions
  # answers = [0 for i in range(0, questions)]

  counts = [0.0 for i in range(0, num_choices)]
  total = 0.0
  # print("question", len(questions))
  model.eval()
  with torch.no_grad():        
    for i, choices in enumerate(questions):
        #print(choices)
        # print(i)
        if i % 1000 == 0 and i != 0:
          print(f"On iteration {i}, accuracy is {counts[0]/total} equal would be {1/num_choices} count is {total}", [i/total for i in counts])
        encoded_choices = [tokenizer.encode(s) for s in choices]
        cls_token_location = [tokens.index(tokenizer.cls_token_id) for tokens in encoded_choices]
        #check if some choices are longer than the others
        max_len = max([len(tokens) for tokens in encoded_choices])
        check = True
        for tokens in encoded_choices:
            if len(tokens) < max_len:
                check = False
                break
        if check:
            input_ids = torch.tensor(encoded_choices).unsqueeze(0)  # Batch size: 1, number of choices: 2
            if len(input_ids) >= 1024:
              continue
            mc_token_ids = torch.tensor([cls_token_location])  # Batch size: 1
            input_ids = input_ids.to(device)
            mc_token_ids = mc_token_ids.to(device)
            outputs = model(input_ids, mc_token_ids=mc_token_ids)
            lm_logits = outputs.logits
            mc_logits = outputs.mc_logits
            counts[mc_logits.argmax().item()] += 1
            total += 1
            model_ans.append(mc_logits.argmax().item())
        else:
          model_ans.append(-1)
        torch.cuda.empty_cache() 
  print(f"FINAL for num_choices {num_choices}: accuracy is {counts[0]/total} equal would be {1/num_choices} count is {total}", [i/total for i in counts])

  # print(counts)
  # print(total)

NUM_CHOICES: 5
On iteration 1000, accuracy is 0.18455366098294884 equal would be 0.2 count is 997.0 [0.18455366098294884, 0.21865596790371114, 0.1855566700100301, 0.1945837512537613, 0.21664994984954863]
On iteration 2000, accuracy is 0.1844758064516129 equal would be 0.2 count is 1984.0 [0.1844758064516129, 0.22731854838709678, 0.19203629032258066, 0.20110887096774194, 0.19506048387096775]
On iteration 3000, accuracy is 0.18737407656145064 equal would be 0.2 count is 2978.0 [0.18737407656145064, 0.2269979852249832, 0.1944257891202149, 0.20785762256548018, 0.18334452652787106]
On iteration 4000, accuracy is 0.19219143576826198 equal would be 0.2 count is 3970.0 [0.19219143576826198, 0.22115869017632242, 0.1889168765743073, 0.20629722921914356, 0.19143576826196473]
On iteration 5000, accuracy is 0.1946795646916566 equal would be 0.2 count is 4962.0 [0.1946795646916566, 0.2158403869407497, 0.19226118500604594, 0.20878677952438532, 0.18843208383716242]
On iteration 6000, accuracy is 0.197

In [ ]:
f = open("compare_5/random_baseline.txt", "w+")
for i in model_ans:
  f.write(str(i)+"\n")
f.close()
print(len(model_ans))

12211


In [ ]:
num_added_tokens = tokenizer.add_special_tokens({"cls_token": "[CLS]"})
# Update the model embeddings with the new vocabulary size
print("tokenizer length", len(tokenizer))
embedding_layer = model.resize_token_embeddings(len(tokenizer))

questions = quiz_questions
# answers = [0 for i in range(0, questions)]

counts = [0.0 for i in range(0, num_choices)]
total = 0.0
print("question", len(questions))
model.eval()      
for i, choices in enumerate(questions):
    #print(choices)
    # print(i)
    if i % 500 == 0 and i != 0:
      print(f"On iteration {i}, accuracy is {counts[0]/total} equal would be {1/num_choices} count is {total}", [i/total for i in counts])
    encoded_choices = [tokenizer.encode(s) for s in choices]
    cls_token_location = [tokens.index(tokenizer.cls_token_id) for tokens in encoded_choices]
    #check if some choices are longer than the others
    max_len = max([len(tokens) for tokens in encoded_choices])
    check = True
    for tokens in encoded_choices:
        if len(tokens) < max_len:
            check = False
            break
    if check:
        input_ids = torch.tensor(encoded_choices).unsqueeze(0)  # Batch size: 1, number of choices: 2
        if len(input_ids) >= 1024:
          continue
        mc_token_ids = torch.tensor([cls_token_location])  # Batch size: 1
        input_ids = input_ids.to(device)
        mc_token_ids = mc_token_ids.to(device)
        with torch.no_grad():  
          outputs = model(input_ids, mc_token_ids=mc_token_ids)
        lm_logits = outputs.logits
        mc_logits = outputs.mc_logits
        counts[mc_logits.argmax().item()] += 1
        total += 1
    torch.cuda.empty_cache() 
print(f"FINAL {i}, accuracy is {counts[0]/total} equal would be {1/num_choices}", [i/total for i in counts])

print(counts)
print(total)

tokenizer length 100259
question 12211
On iteration 500, accuracy is 0.04242424242424243 equal would be 0.04 count is 495.0 [0.04242424242424243, 0.052525252525252523, 0.04242424242424243, 0.04040404040404041, 0.03232323232323232, 0.024242424242424242, 0.046464646464646465, 0.04242424242424243, 0.03636363636363636, 0.044444444444444446, 0.030303030303030304, 0.04242424242424243, 0.044444444444444446, 0.044444444444444446, 0.028282828282828285, 0.03232323232323232, 0.050505050505050504, 0.046464646464646465, 0.030303030303030304, 0.05454545454545454, 0.03434343434343434, 0.044444444444444446, 0.030303030303030304, 0.03636363636363636, 0.046464646464646465]
On iteration 1000, accuracy is 0.04757085020242915 equal would be 0.04 count is 988.0 [0.04757085020242915, 0.043522267206477734, 0.0465587044534413, 0.03441295546558704, 0.03340080971659919, 0.03744939271255061, 0.04757085020242915, 0.04048582995951417, 0.032388663967611336, 0.043522267206477734, 0.04048582995951417, 0.03744939271255

In [ ]:
data = []
for i in range(0, 10):
  print("On trial", i)
  data.append(trial())

In [ ]:
import numpy as np
print(data)
print("mean", np.mean(data))
print("std", np.std(data))

In [ ]:
def trial():
  tokenizer = AutoTokenizer.from_pretrained("gpt2")
  model = GPT2DoubleHeadsModel.from_pretrained("gpt2")
  model = model.to(device)
  # Get the current vocabulary size
  original_vocab_size = len(tokenizer)
  new_word_list = [i.strip().lower() for i in open("words.txt").readlines()]
  num_new_words = len(new_word_list)

  num_added = tokenizer.add_tokens(new_word_list)

  model.resize_token_embeddings(len(tokenizer))
  num_added_tokens = tokenizer.add_special_tokens({"cls_token": "[CLS]"})
  # Update the model embeddings with the new vocabulary size
  embedding_layer = model.resize_token_embeddings(len(tokenizer))

  questions = quiz_questions
  # answers = [0 for i in range(0, questions)]

  count0 = 0.0
  count1 = 0.0
  count2 = 0.0
  count3 = 0.0
  count4 = 0.0
  total = 0.0
  model.eval()
  with torch.no_grad():        
    for i, choices in enumerate(questions):
        #print(choices)
        # print(i)
        # if i % 500 == 0 and i != 0:
        #   print(f"On iteration {i}, accuracy is {count0/total} count1: {count1/total} count2: {count2/total} count3: {count3/total} count4: {count4/total}")
        encoded_choices = [tokenizer.encode(s) for s in choices]
        cls_token_location = [tokens.index(tokenizer.cls_token_id) for tokens in encoded_choices]
        #check if some choices are longer than the others
        max_len = max([len(tokens) for tokens in encoded_choices])
        check = True
        for tokens in encoded_choices:
            if len(tokens) < max_len:
                check = False
        if check:
            input_ids = torch.tensor(encoded_choices).unsqueeze(0)  # Batch size: 1, number of choices: 2
            if len(input_ids) >= 1024:
              continue
            mc_token_ids = torch.tensor([cls_token_location])  # Batch size: 1
            input_ids = input_ids.to(device)
            mc_token_ids = mc_token_ids.to(device)
            outputs = model(input_ids, mc_token_ids=mc_token_ids)
            lm_logits = outputs.logits
            mc_logits = outputs.mc_logits
            if mc_logits.argmax().item() == 0:
                count0 += 1
            elif mc_logits.argmax().item() == 1:
                count1 += 1
            elif mc_logits.argmax().item() == 2:
                count2 += 1
            elif mc_logits.argmax().item() == 3:
                count3 += 1
            elif mc_logits.argmax().item() == 4:
                count4 += 1
            
            total += 1
  print(f"FINAL count0: {count0/total} count1: {count1/total} count2: {count2/total} count3: {count3/total} count4: {count4/total}")
  return count0/total
  # print(len(questions))
  # print(count0)
  # print(count1)
  # print(count2)
  # print(count3)
  # print(count4)
  # print(total)